In [6]:
import sys
import os
from source import plotFunctions
from timeit import default_timer as timer
import pandas as pd
import numpy as np
from experiments import setup
from source import classifiers
from source import metrics
from source import util
from experiments.methods import svm
from experiments.methods import proposed_gmm_core_extraction
from experiments.methods import improved_intersection
from experiments.methods import compose
from experiments.methods import raul_classifier
from sklearn.grid_search import GridSearchCV
#from hpsklearn import HyperoptEstimator, any_classifier
#from hyperopt import tpe
'''
from experiments.methods import compose2
from experiments.methods import compose3
from experiments.methods import intersection
'''

def main():
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'experiments'+sep+'data'+sep
    
    #loading a dataset
    allDataSets = [setup.loadCDT, setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.loadUG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadMG_2C_2D, setup.loadFG_2C_2D, setup.loadGEARS_2C_2D, setup.load4CR, setup.load4CE1CF, setup.loadCheckerBoard]
    sizes=[200]
    batches=[50]
    #for d in range(len(allDataSets)):
    dataValues, dataLabels, databaseName = allDataSets[0](path, sep)
    raul = raul_classifier.raulClassifier()
    #testing grid search
    #, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65
    tuned_params = {"excludingPercentage" : [0.95, 0.7], "clfName":['cl', 'svm', 'rf'], "sizeOfBatch":[sizes[0]], "batches":[batches[0]]}
    gs = GridSearchCV(raul, tuned_params)
    gs.fit(dataValues, dataLabels)
    output = '{} -- Best Score: {}, achieved with following parameters: {}.'.format(databaseName,gs.best_score_,gs.best_params_)
    f = open('output'+sep+'gridsearch_selection.txt', 'a')
    f.write(output)
    f.close()
    
if __name__ == "__main__":
    main()


IndexError: index 200 is out of bounds for axis 0 with size 200